# Spotify scraping notebook

### Imports

Install with: 
```pip install [package1] [package2] ...```

In [1]:
import ast
import datetime
import json
import re
import sys

import pandas as pd
import numpy as np

import spotipy
import spotipy.util as util

In [4]:
# to use with your account, fill in with your username and request own token
username = 'ejonokuchi'
token = 'BQCi5Atyf9H9iIfDAUhAhnf4LtVa-jZvwdD5TLtByB1Wy4T-LeRI9d6lJOmEkgIU4qX57OXL_pn2QyEN4ehiPnOVopmZzYoYkeDjTs-6UqZbCKjbJ6O24ojWj8XzQKzpD_ZWKURBh2-Oj7qzG6Cq74TZohcDvCxaWpZzJxVkgGhAzZRAMIbz_9LsUtv0S-XPL_0MsubuCpxDXM1u0qVUI1peq0NGQbNuALEAUC7_zK8jxsCt9AT8pRWbTzNDKNiCeU8d-ZWOfmOIkNOpJ4A'

In [5]:
# Initialize spotipy client
sp = spotipy.Spotify(auth=token)

For reference, see the [spotipy docs](http://spotipy.readthedocs.io/en/latest/)

In [6]:
def get_spotify_artist_obj(sp, name):
    '''
    Get the artist ID for a spotify artist. Returns the first match.
    '''
    print('Searching Spotify for: \'{}\''.format(name))
    results = sp.search(q=name, type='artist')
    if 'artists' not in results or results['artists']['total'] < 1 or len(results['artists']['items']) < 1:
        print('No results found\n')
        return None
#         raise RuntimeError('No artists found\n')
    first_match = results['artists']['items'][0]
    print('           Found artist \'{}\''.format(first_match['name']))
    print('                Genres: \'{}\'\n'.format(first_match['genres']))
    return first_match

In [7]:
# Example artist: Michael Jackson
query = 'kool gang'
get_spotify_artist_obj(sp, query)

Searching Spotify for: 'kool gang'
           Found artist 'Kool & The Gang'
                Genres: '['classic funk rock', 'disco', 'funk', 'motown', 'post-disco', 'quiet storm', 'soul', 'soul christmas']'



{'external_urls': {'spotify': 'https://open.spotify.com/artist/3VNITwohbvU5Wuy5PC6dsI'},
 'followers': {'href': None, 'total': 542879},
 'genres': ['classic funk rock',
  'disco',
  'funk',
  'motown',
  'post-disco',
  'quiet storm',
  'soul',
  'soul christmas'],
 'href': 'https://api.spotify.com/v1/artists/3VNITwohbvU5Wuy5PC6dsI',
 'id': '3VNITwohbvU5Wuy5PC6dsI',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/60546a9f9a406f1a553601b699aab0e133395d3f',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/db2e107d552846b92658de5a79f48d704ddc136b',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/be286a5dc61cb9f006c457522eee614a3623bef0',
   'width': 160}],
 'name': 'Kool & The Gang',
 'popularity': 66,
 'type': 'artist',
 'uri': 'spotify:artist:3VNITwohbvU5Wuy5PC6dsI'}

<hr>

## Compile all artists

In [8]:
billboard_df = pd.read_csv('billboard_data.csv')
billboard_df

,artist_name,date,rank,year
0,Drake,3/31/18,1,2018
1,Ed Sheeran,3/31/18,2,2018
2,Bruno Mars & Cardi B,3/31/18,3,2018
3,Bebe Rexha & Florida Georgia Line,3/31/18,4,2018
4,Post Malone Featuring Ty Dolla $ign,3/31/18,5,2018
5,"Zedd, Maren Morris & Grey",3/31/18,6,2018
6,Camila Cabello Featuring Young Thug,3/31/18,7,2018
7,The Weeknd & Kendrick Lamar,3/31/18,8,2018
8,BlocBoy JB Featuring Drake,3/31/18,9,2018
9,Kendrick Lamar & SZA,3/31/18,10,2018


300,000 rows, huh. Let's see how many artists.

In [9]:
len(billboard_df['artist_name'].unique())

9096

Not bad. Let's peep the city artists before going any further. Just going to compile the three files, not sure why they're separated.

In [10]:
file1, file2, file3 = 'city_artists_file.txt', 'city_artists_file2.txt', 'city_artists_file3.txt'
city_df = pd.read_csv(file1).append(pd.read_csv(file2)).append(pd.read_csv(file3)).reset_index().drop('index', axis=1)
city_df

,Artists,City,State
0,"['Ray J, Boosie Badazz', 'Ray J, The Mob Group...",Albuquerque,New Mexico
1,[],Alexandria,Virginia
2,"['Shoreline Mafia', 'Shoreline Mafia', 'Shorel...",Anaheim,California
3,"['Sammy Adams', ""Lin-Manuel Miranda, 'In The H...",Ann Arbor,Michigan
4,"['MO3', 'MO3, Moneybagg Yo', 'Mr. Pookie', 'La...",Arlington,Texas
5,"['Ray J, Boosie Badazz', 'Ray J, The Mob Group...",Asheville,North Carolina
6,"['Riley Green', 'Riley Green', 'YFN Lucci', 'Y...",Atlanta,Georgia
7,"['La Maquinaria Norteña', 'Tech N9ne, Snow Tha...",Aurora,Colorado
8,"['David Ramirez', 'Becca Mancari', 'Twain', 'T...",Austin,Texas
9,"['RJMrLA', 'Mike Sherm', 'Frostydasnowmann', '...",Bakersfield,California


Now let's begin by looking at null or blank values.

In [11]:
billboard_df.isnull().sum()

artist_name    0
date           0
rank           0
year           0
dtype: int64

In [12]:
sum(billboard_df.artist_name.apply(lambda x: len(x) < 1))

0

In [13]:
city_df.isnull().sum()

Artists     0
City        0
State      21
dtype: int64

In [14]:
sum(city_df.Artists.apply(lambda x: len(ast.literal_eval(x)) < 1))

252

Let's drop nulls and blanks.

In [15]:
billboard_df = billboard_df.dropna()
city_df = city_df.dropna()
city_df['Artists'] = city_df['Artists'].apply(ast.literal_eval)
city_df = city_df[city_df.Artists.apply(lambda x: len(x) > 0)]

/Users/ejonokuchi/.virtualenvs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Let's make a set of artist names from both dataframes.

In [16]:
# Billboard artists are separated by '&'; ','; 'Featuring'; 'Feat.'; and sometimes '()'
billboard_artist_groups = list(set(billboard_df.artist_name.unique().tolist()))

billboard_pattern = re.compile(r'( With )|( X )|( x )|(/)|( with )|( feat. )|(\))|( featuring )|( Featuring )|( \(featuring)|( Feat. )|( \(Featuring)|(, )')
billboard_pattern_with_and = re.compile(r'( With )|( X )|( x )|(/)|( & )|( with )|( feat. )|(\))|( featuring )|( Featuring )|( \(featuring)|( Feat. )|( \(Featuring)|(, )')
bad_strings = [' With ', ' & ', ' X ', ' x ', '/', ' with ', ' feat. ', ')', ' featuring ', ' Featuring ', ' (featuring', ' Feat. ', ' (Featuring', ', ', None]

# For every artist group, if there is a featuring, we want to include splitting by '&'
# Otherwise, don't split by '&', otherwise we will accidentally split artists like 'Daryl Hall & John Oates'

clean_billboard_artists = []
for artist_group in billboard_artist_groups:
    if 'Feat' in artist_group:
        clean_billboard_artists += [a for a in re.split(billboard_pattern_with_and, artist_group) if a not in bad_strings]
    else:
        clean_billboard_artists += [a for a in re.split(billboard_pattern, artist_group) if a not in bad_strings]
clean_billboard_artists

['Adele',
 'Hank Jacobs',
 'Les Compagnons De La Chanson',
 '2Pac',
 'Notorious B.I.G.',
 'Radio',
 'Dramacydal & Stretch',
 'Republica',
 'Terry Jacks',
 'The Yellow Balloon',
 'Katy Perry',
 'Skip Marley',
 'Jody Miller',
 'Nikki',
 'Reba McEntire Duet',
 'Kelly Clarkson',
 'The Hollywood Stars',
 'Kid Ink',
 'Fetty Wap',
 'Pitbull',
 'Ne-Yo',
 'Afrojack',
 'Nayer',
 'Leon Redbone',
 'Fancy',
 'Al Stewart',
 'Mariah Carey',
 'Joe',
 '98 Degrees',
 'Nino & The Ebb Tides',
 'G-Eazy',
 'Bebe Rexha',
 'Wiz Khalifa',
 'Travi$ Scott',
 'The Wiseguys',
 'Maureen Steele',
 'Kanye West',
 'Young Jeezy',
 'American Idol Finalists',
 'Trilogy',
 'Nappy Roots',
 'Anthony Hamilton',
 'Kanye West',
 'Lupe Fiasco',
 'Paula DeAnda',
 'The DEY',
 'Eminem',
 'Nate Dogg',
 'Orion The Hunter',
 'The Mojo Men',
 'Philip Bailey',
 'Phil Collins',
 'Ariana Grande',
 'Big Sean',
 'Cliff Richard And The Shadows',
 'A Great Big World & Christina Aguilera',
 'Edwyn Collins',
 'Yung Wun',
 'DMX',
 "Lil' Flip",


In [17]:
# Get city artists. Much cleaner, usually only separated by a ','
city_pattern = re.compile(r'(, )')

clean_city_artists = []
for artist_group in city_df.Artists:
    clean_city_artists += [a for artist in artist_group for a in re.split(city_pattern, artist) if a != ', ']
clean_city_artists

['Ray J',
 'Boosie Badazz',
 'Ray J',
 'The Mob Group',
 'Payso B',
 'Pakman Jitt',
 'Alec Joseph',
 'Fox Stevenson',
 'Shaboozey',
 'Tenside',
 'Franz Liszt',
 'Dinu-Mihai Stefan',
 'N3WPORT',
 'MC Ceja',
 'Back Talk',
 'The Fame Riot',
 'Tenside',
 'Sergei Rachmaninoff',
 'Dinu-Mihai Stefan',
 'Alec Joseph',
 'Ludwig van Beethoven',
 'Dinu-Mihai Stefan',
 'Franz Liszt',
 'Dinu-Mihai Stefan',
 'N3WPORT',
 'Annaka',
 'Franz Liszt',
 'Dinu-Mihai Stefan',
 'Ludwig van Beethoven',
 'Dinu-Mihai Stefan',
 'Sergei Rachmaninoff',
 'Dinu-Mihai Stefan',
 'Ludwig van Beethoven',
 'Dinu-Mihai Stefan',
 'Clean Bandit',
 'Zara Larsson',
 'Joseph Duveen',
 'LODATO',
 'Alec Joseph',
 'Chel',
 'The Eiffels',
 'DJ Spinz',
 'K-Major',
 'DJ Nan2',
 'LuiGProdigy',
 'Ashla Taylor',
 'Far Out',
 'B. Howard',
 'Mr. 2Kay',
 'Reekado Banks',
 'Nick Tangorra',
 'Razor B',
 'Nu Born',
 'Bobbi Storm',
 'David Fanning',
 'Bella Dose',
 'Shifta',
 'Wayne Wonder',
 'Clean Bandit',
 'Sean Paul',
 'Anne-Marie',
 'Jose

In [18]:
# ignore whitespace and one letter artists
all_artists = [x for x in list(set(clean_city_artists + clean_billboard_artists)) if len(x) > 1]
all_artists

['Jessie Booth',
 'Tinie Tempah',
 'Greedo',
 'Adele',
 'Hank Jacobs',
 'Les Compagnons De La Chanson',
 'Jellybean',
 'Republica',
 'Terry Jacks',
 'Ishmael Raps',
 'The Yellow Balloon',
 'Jody Miller',
 'Nikki',
 'The Hollywood Stars',
 'Yavahn',
 'Leon Redbone',
 'Fancy',
 'The Expendables',
 'Al Stewart',
 'Nino & The Ebb Tides',
 'The Wiseguys',
 'Luis y Julian',
 'Maureen Steele',
 'American Idol Finalists',
 'Trilogy',
 'La Diferenzia',
 'G $ Money',
 'Zack Knight',
 'Jameson Rodgers',
 'Autograf',
 'Orion The Hunter',
 'The Mojo Men',
 'Tom Petty & The Heartbreakers',
 'Cliff Richard And The Shadows',
 'A Great Big World & Christina Aguilera',
 'Edwyn Collins',
 'Lauren Jauregui',
 'Markell',
 'Ty Money',
 'Galactic',
 'The Fortunes',
 'Big Quis',
 'Macrointelligence',
 'JAY Z',
 'John Anderson',
 'Oh Wonder',
 'LaToya London',
 'Young Mezzy',
 'The Pasadenas',
 'Spencer Ross',
 'VAVO',
 'Anna Kendrick',
 'Anthony Stewart',
 'The Greenwood County Singers',
 'PIKOTARO',
 'Ty Pop

How many we got?

In [19]:
len(all_artists)

13200

Let's write this into the artists file now.

In [20]:
# # don't re-run this!
# with open('all_artists.txt', mode='w') as o:
#     o.write('\n'.join(sorted(all_artists)))

<hr>

## Get data from Spotify

Let's get crankin. First, make sure we save our progress everytime this next cell runs.

In [21]:
# Make sure to log everything we do
log_file = 'artist_scrape_log.txt'
artist_genres_file = 'artists_genres.csv'
artists_file = 'all_artists.txt'


with open(log_file, "a") as logger:

    def log(msg):
        '''Print and log a message'''
        print(msg)
        logger.write(msg + '\n')

    def get_spotify_artist_obj(sp, name):
        '''Get the artist ID for a spotify artist. Returns the first match.'''
        log('Searching Spotify for: \'{}\''.format(name))
        results = sp.search(q=name, type='artist')
        if 'artists' not in results or results['artists']['total'] < 1 or len(results['artists']['items']) < 1:
            log('No results found.\n')
            return None
            # raise RuntimeError('No artists found\n')
        first_match = results['artists']['items'][0]
        log('         Found artist: \'{}\''.format(first_match['name']))
        log('               Genres: \'{}\'\n'.format(first_match['genres']))
        return first_match
    
    log('*' * 80 + '\nStarting session at ' + str(datetime.datetime.now()) + '\n' + '*' * 80 + '\n')

    # Pickup where we left off
    num_artists = 0
    with open(artists_file) as i:
        all_artists = i.read().split('\n')
        log('Reading artists file...')
        num_artists = len(all_artists)
        log('Found {} artists'.format(num_artists))

    log('Reading artist genres file...')
    genres_df = pd.read_csv(artist_genres_file)
    
    completed_artists = set(genres_df['read_name'].tolist())
    start_num_completed_artists = len(completed_artists)
    log('Found {} searched artists'.format(start_num_completed_artists))
    
    start_num_filled_out_artists = sum(genres_df.genres.apply(lambda x: len(ast.literal_eval(x)) > 0))
    log('Found {} artists with non-empty genres'.format(start_num_filled_out_artists))
    
    end_num_completed_artists = start_num_completed_artists
    end_num_filled_out_artists = start_num_filled_out_artists
    log('Searching Spotify for artists...')
    

    # First, try to just get any artists not searched yet
    artist_object_list = []
    try:
        for artist in all_artists:
            if artist not in completed_artists:
                new_artist = get_spotify_artist_obj(sp, artist)
                if new_artist is None:
                    artist_object_list.append({
                        'read_name': artist,
                        'found_name': None,
                        'spotify_id': None,
                        'genres': [],
                    })
                else:
                    artist_object_list.append({
                        'read_name': artist,
                        'found_name': new_artist['name'],
                        'spotify_id': new_artist['id'],
                        'genres': new_artist['genres'],
                    })
                    end_num_filled_out_artists += 1
                end_num_completed_artists += 1
                completed_artists.add(artist)
    except Exception as err:
        # Log exception and update dataframe
        log('Encountered an error:')
        log(str(err) + '\n')
    
    genres_df = genres_df.append(pd.DataFrame(artist_object_list)).reset_index().drop('index', axis=1)

    # TODO: get the genres for all of the Spotify artists

    # Save progress
    log('Progress Report:')
    log('            Starting artists completed: {}'.format(start_num_completed_artists))
    log('              Ending artists completed: {}'.format(end_num_completed_artists))
    log('Starting artists with non-empty genres: {}'.format(start_num_filled_out_artists))
    log('  Ending artists with non-empty genres: {}'.format(end_num_filled_out_artists))
    
    # Write to csv and artists file
    genres_df.to_csv(artist_genres_file, index=False)
    with open(artists_file, mode='w') as o:
        o.write('\n'.join(sorted(all_artists)))

    log('\n' + '*' * 80 + '\nEnding session at ' + str(datetime.datetime.now()) + '\n' + '*' * 80 + '\n\n\n')

********************************************************************************
Starting session at 2018-04-16 06:14:36.564185
********************************************************************************

Reading artists file...
Found 13200 artists
Reading artist genres file...
Found 8622 searched artists
Found 5080 artists with non-empty genres
Searching Spotify for artists...
Searching Spotify for: ' & The Mysterians'
         Found artist: '? & The Mysterians'
               Genres: '['bubblegum pop', 'classic garage rock', 'freakbeat', 'garage rock', 'protopunk', 'psychedelic rock']'

Searching Spotify for: ' 702'
         Found artist: '702'
               Genres: '['hip pop', 'neo soul', 'new jack swing', 'quiet storm', 'r&b', 'southern hip hop', 'urban contemporary']'

Searching Spotify for: ' Aaron Neville'
         Found artist: 'Aaron Neville'
               Genres: '['jazz blues', 'jazz christmas', 'memphis soul', 'motown', 'new orleans blues', 'pop christmas', 'quiet s

         Found artist: ''Legally Blonde' Ensemble'
               Genres: '[]'

Searching Spotify for: ''Lgado'
         Found artist: 'Lgado'
               Genres: '[]'

Searching Spotify for: ''Something Rotten' Ensemble'
         Found artist: ''Something Rotten' Ensemble'
               Genres: '[]'

Searching Spotify for: '(+44'
         Found artist: '+44'
               Genres: '['emo', 'pop punk']'

Searching Spotify for: '03 Greedo'
         Found artist: '03 Greedo'
               Genres: '['vapor trap', 'west coast trap']'

Searching Spotify for: '070 Shake'
         Found artist: '070 Shake'
               Genres: '['trap soul', 'underground hip hop']'

Searching Spotify for: '100 Strings and Jono (Choir of 40 Voices'
No results found.

Searching Spotify for: '1017 Eskimo'
         Found artist: '1017 Eskimo'
               Genres: '[]'

Searching Spotify for: '11/5'
         Found artist: '11/5'
               Genres: '['g funk', 'hyphy']'

Searching Spotify for: '11:11'


         Found artist: 'Ab-Soul'
               Genres: '['alternative hip hop', 'gangster rap', 'hip hop', 'indie r&b', 'pop rap', 'rap', 'southern hip hop', 'trap music', 'underground hip hop']'

Searching Spotify for: 'Abe Kaelin'
         Found artist: 'Abe Kaelin'
               Genres: '[]'

Searching Spotify for: 'Abhi The Nomad'
         Found artist: 'Abhi The Nomad'
               Genres: '['vapor soul']'

Searching Spotify for: 'Abraham & Bethliza'
         Found artist: 'Abraham & Bethliza'
               Genres: '[]'

Searching Spotify for: 'Abraham Velazquez'
         Found artist: 'Abraham Velazquez'
               Genres: '['latin christian']'

Searching Spotify for: 'Ace Cino'
         Found artist: 'Ace Cino'
               Genres: '[]'

Searching Spotify for: 'Ace NumbaFive'
         Found artist: 'Ace NumbaFive'
               Genres: '[]'

Searching Spotify for: 'Acid Tongue'
         Found artist: 'Acid Tongue'
               Genres: '[]'

Searching Spotify for: '

         Found artist: 'Alexander Schreiner'
               Genres: '[]'

Searching Spotify for: 'Alexandre Desplat'
         Found artist: 'Alexandre Desplat'
               Genres: '['scorecore', 'soundtrack']'

Searching Spotify for: 'Alexio'
         Found artist: 'Alexio'
               Genres: '['reggaeton flow']'

Searching Spotify for: 'Alexis La Bestia'
         Found artist: 'Alexis La Bestia'
               Genres: '[]'

Searching Spotify for: 'Alexis y Fido'
         Found artist: 'Alexis y Fido'
               Genres: '['latin', 'latin hip hop', 'reggaeton', 'reggaeton flow', 'trap latino', 'tropical']'

Searching Spotify for: 'Alextbh'
         Found artist: 'Alextbh'
               Genres: '['deep indie r&b', 'indie r&b', 'malaysian indie']'

Searching Spotify for: 'Alfredo Castañeda'
         Found artist: 'Alfredo Castañeda'
               Genres: '[]'

Searching Spotify for: 'Alfredo Olivas'
         Found artist: 'Alfredo Olivas'
               Genres: '['deep region

         Found artist: 'Andrea Peña'
               Genres: '[]'

Searching Spotify for: 'Andreas Moss'
         Found artist: 'Andreas Moss'
               Genres: '[]'

Searching Spotify for: 'Andres "el Jibaro" Jimenez'
         Found artist: 'Andrés Jiménez, El Jíbaro'
               Genres: '['puerto rican rock']'

Searching Spotify for: 'Andrew Bird'
         Found artist: 'Andrew Bird'
               Genres: '['art pop', 'chamber pop', 'chicago indie', 'folk christmas', 'folk-pop', 'freak folk', 'indie christmas', 'indie folk', 'indie pop', 'indie rock', 'melancholia', 'modern rock', 'new americana', 'singer-songwriter', 'stomp and holler']'

Searching Spotify for: 'Andrew Lloyd Webber'
         Found artist: 'Andrew Lloyd Webber'
               Genres: '['broadway', 'hollywood', 'show tunes']'

Searching Spotify for: 'Andrew McMahon in the Wilderness'
         Found artist: 'Andrew McMahon in the Wilderness'
               Genres: '['indie pop', 'indie poptimism', 'indietronica

...

         Found artist: 'Vampire Weekend'
               Genres: '['brooklyn indie', 'folk-pop', 'garage rock', 'indie folk', 'indie pop', 'indie rock', 'indietronica', 'modern rock', 'neo-psychedelic', 'synthpop']'

Searching Spotify for: 'Van Alden'
         Found artist: 'Van Alden'
               Genres: '[]'

Searching Spotify for: 'Vancouver Sleep Clinic'
         Found artist: 'Vancouver Sleep Clinic'
               Genres: '['chamber pop', 'indie anthem-folk', 'indie folk', 'indie r&b', 'vapor pop', 'vapor soul']'

Searching Spotify for: 'Vanity Strikes'
         Found artist: 'Vanity Strikes'
               Genres: '[]'

Searching Spotify for: 'Vaughan Mason and Crew'
         Found artist: 'Vaughan Mason and Crew'
               Genres: '[]'

Searching Spotify for: 'Vayven Del Amor'
         Found artist: 'El Vayven Del Amor'
               Genres: '[]'

Searching Spotify for: 'Vegas Ultra Lords'
         Found artist: 'Vegas Ultra Lords'
               Genres: '[]'

Searching

         Found artist: 'Waterloo Revival'
               Genres: '['contemporary country', 'lift kit']'

Searching Spotify for: 'Watsky'
         Found artist: 'Watsky'
               Genres: '['abstract hip hop', 'comic', 'indie pop rap', 'pop rap']'

Searching Spotify for: 'Wayne Toups'
         Found artist: 'Wayne Toups'
               Genres: '['cajun', 'swamp pop', 'zydeco']'

Searching Spotify for: 'Wayne Wilcox'
         Found artist: 'Wayne Wilcox'
               Genres: '[]'

Searching Spotify for: 'Wayne Wilcox (Gordon)'
         Found artist: 'Wayne Wilcox (Gordon)'
               Genres: '[]'

Searching Spotify for: 'We Are Messengers'
         Found artist: 'We Are Messengers'
               Genres: '['ccm', 'christian alternative rock', 'christian music', 'christian rock', 'worship']'

Searching Spotify for: 'We The Kings'
         Found artist: 'We The Kings'
               Genres: '['emo', 'modern rock', 'neo mellow', 'piano rock', 'pop punk', 'pop rock', 'post-teen po

         Found artist: 'Wire'
               Genres: '['alternative pop', 'alternative rock', 'art pop', 'art rock', 'chamber psych', 'dance rock', 'dance-punk', 'dream pop', 'experimental', 'experimental rock', 'fourth world', 'garage psych', 'garage rock', 'gbvfi', 'indie rock', 'industrial', 'industrial rock', 'jangle pop', 'lo-fi', 'math rock', 'neo-psychedelic', 'new wave', 'no wave', 'noise pop', 'noise rock', 'nu gaze', 'post-hardcore', 'post-punk', 'power pop', 'psychedelic rock', 'punk', 'punk blues', 'shoegaze', 'singer-songwriter', 'slow core', 'uk post-punk']'

Searching Spotify for: 'Wiso G'
         Found artist: 'Wiso G'
               Genres: '['latin hip hop']'

Searching Spotify for: 'Witt Lowry'
         Found artist: 'Witt Lowry'
               Genres: '['indie pop rap']'

Searching Spotify for: 'Wolf Alice'
         Found artist: 'Wolf Alice'
               Genres: '['alt-indie rock', 'garage rock', 'indie rock', 'indietronica', 'modern rock', 'shimmer pop']'

Sear

         Found artist: 'Yoey Composes'
               Genres: '[]'

Searching Spotify for: 'Yofrangel'
         Found artist: 'Yofrangel'
               Genres: '['merengue urbano']'

Searching Spotify for: 'Yoga Music'
         Found artist: 'Yoga Music'
               Genres: '[]'

Searching Spotify for: 'Yoga Workout Music'
         Found artist: 'Yoga Workout Music'
               Genres: '[]'

Searching Spotify for: 'Yomil y El Dany'
         Found artist: 'Yomil y El Dany'
               Genres: '['cubaton']'

Searching Spotify for: 'Yomo'
         Found artist: 'Yomo'
               Genres: '['latin', 'latin hip hop', 'reggaeton', 'reggaeton flow', 'trap latino', 'tropical']'

Searching Spotify for: 'Yonder Dale'
         Found artist: 'Yonder Dale'
               Genres: '[]'

Searching Spotify for: 'Yonder Mountain String Band'
         Found artist: 'Yonder Mountain String Band'
               Genres: '['bluegrass', 'jam band', 'new americana', 'progressive bluegrass', 'roots

         Found artist: 'Zion I'
               Genres: '['abstract hip hop', 'alternative hip hop', 'underground rap']'

Searching Spotify for: 'Zion.T'
         Found artist: 'Zion.T'
               Genres: '['k-indie', 'k-pop', 'korean pop']'

Searching Spotify for: 'Ziv Zaifman'
         Found artist: 'Ziv Zaifman'
               Genres: '[]'

Searching Spotify for: 'Zoey Dollaz'
         Found artist: 'Zoey Dollaz'
               Genres: '['dwn trap', 'trap music']'

Searching Spotify for: 'Zomboy'
         Found artist: 'Zomboy'
               Genres: '['brostep', 'catstep', 'complextro', 'edm', 'electro house', 'electronic trap', 'filthstep', 'glitch', 'moombahton', 'zapstep']'

Searching Spotify for: 'Zoogma'
         Found artist: 'Zoogma'
               Genres: '['jam band']'

Searching Spotify for: 'Zuleyka Barreiro'
         Found artist: 'Zuleyka Barreiro'
               Genres: '['latin christian']'

Searching Spotify for: 'Zydecajun'
         Found artist: 'Zydecajun'
   

Let's also make sure we create a list of available Spotify genres.

In [45]:
# number of artists without a genre
# genres_df = pd.read_csv(artist_genres_file)
print(genres_df.shape[0] - genres_df.genres.value_counts()[0])

7835


In [46]:
all_genres = []
for genre_group in genres_df.genres.apply(ast.literal_eval):
    all_genres += [g for genre in genre_group for g in genre_group]
all_genres = sorted(list(set(all_genres)))
all_genres

['a cappella',
 'abstract beats',
 'abstract hip hop',
 'acid house',
 'acid idm',
 'acid jazz',
 'acid techno',
 'acoustic blues',
 'acoustic pop',
 'adult standards',
 'african gospel',
 'african percussion',
 'african rock',
 'afrobeat',
 'afrobeats',
 'afropop',
 'albanian hip hop',
 'album rock',
 'albuquerque indie',
 'alt-indie rock',
 'alternative americana',
 'alternative ccm',
 'alternative country',
 'alternative dance',
 'alternative emo',
 'alternative hip hop',
 'alternative metal',
 'alternative pop',
 'alternative pop rock',
 'alternative r&b',
 'alternative rock',
 'alternative roots rock',
 'ambeat',
 'ambient',
 'ambient psychill',
 'anarcho-punk',
 'anime',
 'anime score',
 'anthem emo',
 'anthem worship',
 'anti-folk',
 'antiviral pop',
 'appalachian folk',
 'arab folk',
 'arab pop',
 'argentine hip hop',
 'argentine indie',
 'argentine reggae',
 'argentine rock',
 'armenian folk',
 'art pop',
 'art rock',
 'athens indie',
 'atmospheric post rock',
 'aussietronica'

How many genres do we have?

In [47]:
len(all_genres)

957

In [48]:
genre_file = 'all_genres.txt'

In [49]:
with open(genre_file, mode='w') as o:
    o.write('\n'.join(sorted(all_genres)))